# Fantasy Football ReAct Agent
Uses tools to query 2 different embeded draft rankings and select the best player.
The agent has been equiped with a system prompt with a draft strategy. However, this as well as the documents to embed can be customized to suite your personal strategy.

In the end ,the players selected last year earned me 3rd place in my league. This quick project demonstrates the ability for agents to synthesize large amounts of information in a timely manner

Credit for cheat sheets: https://www.fantasypros.com/nfl/tools/cheat-sheets/2024-fantasy-football-cheat-sheet.php
reddit.com/user/WeightLiftingLulu/
data from: fantasypros (rankings), fftoolbox, ftnfantasy, pff.com,sharpfootballanalysis,.com , draftsharks.com sharrpfootballanalysis, profootballnetwork, ftnfantasy,  fanduel/draftkings


In [ ]:
%env OPENAI_API_KEY="your_api_key"

In [ ]:
#install dependencies
%pip install llama-index-llms-openai
%pip install llama-index-readers-file
%pip install llama-index-embeddings-openai

In [3]:
#import dependencies
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)

from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI

In [4]:

#load indexes from storage(if they exist)
try:
    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/CheatSheet_1"
    )
    CheatSheet_1_index = load_index_from_storage(storage_context)

    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/CheatSheet_2"
    )
    CheatSheet_2_index = load_index_from_storage(storage_context)

    

    index_loaded = True
except:
    index_loaded = False

In [5]:

#create data directory if it doesn't exist(only needed once)
!mkdir -p 'data/Fantasy-Football-2024/'

In [5]:
#if the indexes don't exist, create them
if not index_loaded:
    # load data
    CheatSheet_1 = SimpleDirectoryReader(
        input_files=["data/Fantasy-Football-2024/CheatSheet_1.pdf"]
    ).load_data()
    CheatSheet_2 = SimpleDirectoryReader(
        input_files=["data/Fantasy-Football-2024/CheatSheet_2.pdf"]
    ).load_data()
    

    # build index
    CheatSheet_1_index = VectorStoreIndex.from_documents(CheatSheet_1)
    CheatSheet_2_index = VectorStoreIndex.from_documents(CheatSheet_2)
    

    # persist index
    CheatSheet_1_index.storage_context.persist(persist_dir="./storage/CheatSheet_1")
    CheatSheet_2_index.storage_context.persist(persist_dir="./storage/CheatSheet_2")



In [5]:
#create query engines
Cheatsheet_1_engine = CheatSheet_1_index.as_query_engine(similarity_top_k=3)
CheatSheet_2_engine = CheatSheet_2_index.as_query_engine(similarity_top_k=3)

In [6]:

#create query engine tools
query_engine_tools = [
    QueryEngineTool(
        query_engine=Cheatsheet_1_engine,
        metadata=ToolMetadata(
            name="Position_Rankings",
            description=(
                "Provides information about position rankings for each fottball position. Can be used to compare players"
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=CheatSheet_2_engine,
        metadata=ToolMetadata(
            name="2024_Fantasy_Football_Cheatsheet",
            description=(
                "Provides detailed information about players. USe to find the best player to draft."
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
    
]

In [7]:

context ="""You are an expert at fantasy football drafts.
 You win your league ever year and your in a draft right now.
You need to use the tools in an intellegent way to select the best player or answer the question provided.
Use both tools and explain your answer. Always use tools.

A round, roster, and available players will be provided.
 Compare this roster to the draft strategy to determine which position is needed.
Use the Position_Ranking Tool to identify the 3 best available player at that position.
Then use the 2024_Fantasy_Football_Cheatsheet tool to look for the three best players.
Finally, form a report on your findings. In the report include multiple options for players to draft.







Fantasy Football Drafting Strategy
**1. Stay Flexible:
Be ready for unexpected twists in the draft. Have multiple strategies in mind so you can adapt on the fly.

2. Focus on Positional Value:

WR and RB Priority: Target WRs and RBs in the first three rounds to build a solid foundation.
Avoid Early TE/QB: Skip drafting a TE or QB in the first three rounds, as there are usually better value options available later.
**3. Utilize Positional Supply and Demand:

Understand the Market: Consider how many players you need at each position and how many are available.
Mispriced Assets: Identify and target players who are undervalued, allowing you to spend less on certain positions early and still get quality later.
4. Draft Capital Allocation:

Balance Your Investment: If you spend early picks on RBs, focus on WRs and Flex positions in the mid-rounds, and vice versa.
Practice Mock Drafts: Rehearse different scenarios using mock drafts to prepare for unexpected events during the actual draft.
5. Preferred Strategies:

Hero RB: Draft one RB and two WRs in the first three rounds, then focus on WRs and TE in Rounds 4-6, and fill out RB/QB in Rounds 7-9.
Super Hero RB: If the RB value is too good to pass up, consider taking two RBs in the first three rounds, then adjust based on how your league typically drafts.
6. Do’s and Don’ts:

Do: Focus on WR and RB in the first three rounds.
Don’t: Take a TE or QB early; avoid using Heavy RB unless there’s exceptional value.
Do: Round out your roster with solid WR/RB picks in Rounds 4-6.
Don’t: Pick both a QB and TE in Rounds 3-6; diversify your selections."""
llm = OpenAI(model="gpt-4")

agent = ReActAgent.from_tools(
    query_engine_tools,
    llm=llm,
    verbose=True,
    context=context
)

In [8]:
#EACH ROUND THE PROMPT MUST BE CHANGED TO REFLECT:
#The round number
#The available players
#The roster
#optional: input of player selection




response = agent.chat("""Its Round 12.
    Available Players: 
            Devin Singletary
Jakobi Meyers
Mike Williams
Jake Ferguson
Jonathon Brooks
Marquise Brown
Tyler Lockett
DeAndre Hopkins
Dallas Goedert
Brock Bowers
T.J. Hockenson
J.K. Dobbins
    Roster:    
        QB: Brock Purdy
        RB: Travis Etienne Jr.
        RB: Joe Mixon
        WR: Justin Jefferson
        WR: Drake London
        TE: Jake Fergeson
        Flex: Davante Adams
        D/ST:
        K:
        BE: Aaron Jones
        BE: Tee Higgins
        BE: Jackson Smith-Njigba
        BE: Jayden Reed
        BE:
        BE:
        BE:           """)
print(str(response))

Thought: Based on the current roster, the team has a solid foundation of RBs and WRs. However, there are still two bench spots and a D/ST spot that need to be filled. Given the draft strategy, it would be wise to fill the bench spots first with the best available players, regardless of their position. Then, we can focus on the D/ST spot in the later rounds.
Action: Position_Rankings
Action Input: {'input': 'Devin Singletary, Jakobi Meyers, Mike Williams, Jake Ferguson, Jonathon Brooks, Marquise Brown, Tyler Lockett, DeAndre Hopkins, Dallas Goedert, Brock Bowers, T.J. Hockenson, J.K. Dobbins'}
Observation: Devin Singletary, Jakobi Meyers, Mike Williams, Jake Ferguson, Jonathon Brooks, Marquise Brown, Tyler Lockett, DeAndre Hopkins, Dallas Goedert, Brock Bowers, T.J. Hockenson, and J.K. Dobbins are all players in the 2024 Fantasy Football Cheat Sheet.
Thought: The Position_Rankings tool didn't provide the expected output. It seems there was a misunderstanding. I was expecting a ranking o